# <span style='font-family:Segoe UI'> **Diseño y construcción de bases de datos**
<span style='font-family:Segoe UI'> Bibiana Quintero Alonso
    
## <span style='font-family:Segoe UI'> **1. Bases de datos**
    
## <span style='font-family:Segoe UI'> **1.1 World database**
    
<span style='font-family:Segoe UI'> Esta base de datos está constituida por 3 tablas:

<span style='font-family:Segoe UI'> **Country:** Posee 239 registros con información de diversos países, así como datos de características económicas y socio-políticas. 
    
<span style='font-family:Segoe UI'> **City:** Posee 4079 registros de las ciudades principales de los países.
    
<span style='font-family:Segoe UI'> **Countrylanguage:** Posee 984 registros con los idiomas de los paises.


In [9]:
source /home/bibianaq/Documentos/Bigdata/world.sql;

Query OK

In [32]:
use world;

Query OK

In [11]:
show tables;

Tables_in_world
city
country
countrylanguage


In [5]:
describe country;

Field,Type,Null,Key,Default,Extra
Code,char(3),NO,PRI,,
Name,char(52),NO,,,
Continent,"enum('Asia','Europe','North America','Africa','Oceania','Antarctica','South America')",NO,,Asia,
Region,char(26),NO,,,
SurfaceArea,"decimal(10,2)",NO,,0.00,
IndepYear,smallint(6),YES,,NULL,
Population,int(11),NO,,0,
LifeExpectancy,"decimal(3,1)",YES,,NULL,
GNP,"decimal(10,2)",YES,,NULL,
GNPOld,"decimal(10,2)",YES,,NULL,


In [14]:
describe city;

Field,Type,Null,Key,Default,Extra
ID,int(11),NO,PRI,NULL,auto_increment
Name,char(35),NO,,,
CountryCode,char(3),NO,MUL,,
District,char(20),NO,,,
Population,int(11),NO,,0,


In [6]:
describe countrylanguage;

Field,Type,Null,Key,Default,Extra
CountryCode,char(3),NO,PRI,,
Language,char(30),NO,PRI,,
IsOfficial,"enum('T','F')",NO,,F,
Percentage,"decimal(4,1)",NO,,0.0,


## <span style='font-family:Segoe UI'> **Consultas SQL**

<span style='font-family:Segoe UI'> **Indicadores socio-económicos de 10 países de Europa**

In [16]:
select Code, Name,
       (Population/SurfaceArea) as 'Densidad',
       round((GNP/Population)*1000000) as 'GNP per capita',
       LifeExpectancy
from country
where Continent like 'Europe'
limit 10;

Code,Name,Densidad,GNP per capita,LifeExpectancy
ALB,Albania,118.3108,942,71.6
AND,Andorra,166.6667,20897,83.5
AUT,Austria,96.4929,26182,77.7
BEL,Belgium,335.5069,24388,77.8
BGR,Bulgaria,73.7959,1487,70.9
BIH,Bosnia and Herzegovina,77.5827,715,71.5
BLR,Belarus,49.3064,1340,68.0
CHE,Switzerland,173.4425,36936,79.6
CZE,Czech Republic,130.3236,5353,74.5
DEU,Germany,230.1390,25965,77.4


<span style='font-family:Segoe UI'> **Países por continente**

In [18]:
select Continent, count(*) as 'Cantidad de paises'
from country
group by Continent;

Continent,Cantidad de paises
Asia,51
Europe,46
North America,37
Africa,58
Oceania,28
Antarctica,5
South America,14


<span style='font-family:Segoe UI'> **Ranking de países con mayor diversidad de idiomas**

In [20]:
select Name, n as 'Número de idiomas'
from country
join 
(  select CountryCode as Code, count(*) as n
   from countrylanguage
   group by CountryCode
) as t1 using(Code)
order by n desc
limit 5;

Name,Número de idiomas
United States,12
Russian Federation,12
India,12
China,12
Canada,12


<span style='font-family:Segoe UI'> **Proporción de países con idioma oficial inglés**

In [23]:
set @n_countries=(select count(*) from countrylanguage);
select concat(round((n_spa/@n_countries)*100, 2), ' %') as Percentage
from 
(
    select count(*) as n_spa
    from countrylanguage
    where IsOfficial='T' and Language='English'
) as t1
;

Percentage
4.47 %


<span style='font-family:Segoe UI'> **Proporción de población mundial que habla alemán**

In [33]:
set @world_pop=(select sum(Population) from country);

select concat(round(100*sum(spa_pop)/@world_pop, 2), ' %') as Percentage
from
(
    select Code, (Population*Percentage/100) as spa_pop
    from country
    join
    (
        select CountryCode as Code, Percentage
        from countrylanguage
        where Language='German'
    ) as t1 using(Code)
) as t2 
;

Percentage
1.52 %


## <span style='font-family:Segoe UI'> **1.2 Sakila database**
    
<span style='font-family:Segoe UI'> Contiene información de una empresa de alquiler de pelícualas en DVD, presenta 16 tablas con datos relacionados a películas, transacciones, empleados, clientes, inventario y tiendas. 



In [1]:
source /home/bibianaq/Documentos/Bigdata/sakila-db/sakila-schema.sql;

Query OK

In [2]:
source /home/bibianaq/Documentos/Bigdata/sakila-db/sakila-data.sql;

Query OK

In [3]:
use sakila;

Query OK

In [35]:
show tables;

Tables_in_sakila
actor
actor_info
address
category
city
country
customer
customer_list
film
film_actor


In [36]:
describe store;

Field,Type,Null,Key,Default,Extra
store_id,tinyint(3) unsigned,NO,PRI,NULL,auto_increment
manager_staff_id,tinyint(3) unsigned,NO,UNI,NULL,
address_id,smallint(5) unsigned,NO,MUL,NULL,
last_update,timestamp,NO,,current_timestamp(),on update current_timestamp()


In [37]:
describe language;

Field,Type,Null,Key,Default,Extra
language_id,tinyint(3) unsigned,NO,PRI,NULL,auto_increment
name,char(20),NO,,NULL,
last_update,timestamp,NO,,current_timestamp(),on update current_timestamp()


## <span style='font-family:Segoe UI'> **Consultas SQL**

<span style='font-family:Segoe UI'> **Géneros con mayor cantidad de películas**

In [38]:
select *
from (
    select c.name as 'Género', count(*) as 'Número de películas'
    from film_category
    join category c using(category_id)
    group by c.name
) as t
order by `Número de películas` desc
limit 5
;

Género,Número de películas
Sports,74
Foreign,73
Family,69
Documentary,68
Animation,66


<span style='font-family:Segoe UI'> **Géneros de películas más vendidos**

In [39]:
select c.name as 'Género', sum(p.amount) as 'Ventas'
from payment p 
join rental r using(rental_id)
join inventory i using(inventory_id)
join film_category f using(film_id)
join category c using(category_id)
group by c.name
order by Ventas desc
limit 5;

Género,Ventas
Sports,5314.21
Sci-Fi,4756.98
Animation,4656.30
Drama,4587.39
Comedy,4383.58


<span style='font-family:Segoe UI'> **Actores y actrices con más actuaciones**

In [42]:
select concat(first_name, ' ', last_name) as 'Actor/Actriz',
       n as 'Actuaciones en películas'
from actor
join
(
    select actor_id, count(film_id) as n from film_actor group by actor_id
) as t1 using(actor_id)
order by n desc
limit 5
;

Actor/Actriz,Actuaciones en películas
GINA DEGENERES,42
WALTER TORN,41
MARY KEITEL,40
MATTHEW CARREY,39
SANDRA KILMER,37


<span style='font-family:Segoe UI'> **Género de las películas en el que han participado los actores y actrices con mayores actuaciones**

In [43]:
with top as (
    select actor_id, count(film_id) as n from film_actor group by actor_id order by n desc limit 5
), top_actor_film as (
    select actor_id, film_id
    from film_actor
    join top using(actor_id)
), top_actor_cat_film as (
    select ta.actor_id, ta.film_id, c.name
    from top_actor_film ta
    join film_category using(film_id) join category c using(category_id)
), result as (
    select actor_id, name, max(n) as appearances
    from (
        select actor_id, name, count(*) as n from top_actor_cat_film group by actor_id, name
    ) as t
    group by actor_id
)

select concat(a.first_name, ' ', a.last_name) as 'Actor/Actriz',
       r.name as 'Género',
       concat(round((r.appearances/top.n)*100, 2), ' %') as 'Participación'
from result r
join top using(actor_id)
join actor a using(actor_id)
;

Actor/Actriz,Género,Participación
GINA DEGENERES,Action,16.67 %
WALTER TORN,Action,12.20 %
MARY KEITEL,Action,10.00 %
MATTHEW CARREY,Action,12.82 %
SANDRA KILMER,Action,10.81 %


## <span style='font-family:Segoe UI'> **1.3 Employees database**
    
<span style='font-family:Segoe UI'> Esta base de datos contiene 8 tablas con información de departamentos donde trabajan empleados, empleados, salarios y cargos relacionados.


In [2]:
source /home/bibianaq/Documentos/Bigdata/test_db-master/employees.sql;

Query OK

In [3]:
use employees;

Query OK

In [64]:
show tables;

Tables_in_employees
current_dept_emp
departments
dept_emp
dept_emp_latest_date
dept_manager
employees
salaries
titles


In [56]:
describe employees;

Field,Type,Null,Key,Default,Extra
emp_no,int(11),NO,PRI,NULL,
birth_date,date,NO,,NULL,
first_name,varchar(14),NO,,NULL,
last_name,varchar(16),NO,,NULL,
gender,"enum('M','F')",NO,,NULL,
hire_date,date,NO,,NULL,


## <span style='font-family:Segoe UI'> **Consultas SQL**

<span style='font-family:Segoe UI'> **10 registros de empleados**


In [72]:
select * from employees limit 10;

emp_no,birth_date,first_name,last_name,gender,hire_date
10001,1953-09-02,Georgi,Facello,M,1986-06-26
10002,1964-06-02,Bezalel,Simmel,F,1985-11-21
10003,1959-12-03,Parto,Bamford,M,1986-08-28
10004,1954-05-01,Chirstian,Koblick,M,1986-12-01
10005,1955-01-21,Kyoichi,Maliniak,M,1989-09-12
10006,1953-04-20,Anneke,Preusig,F,1989-06-02
10007,1957-05-23,Tzvetan,Zielinski,F,1989-02-10
10008,1958-02-19,Saniya,Kalloufi,M,1994-09-15
10009,1952-04-19,Sumant,Peac,F,1985-02-18
10010,1963-06-01,Duangkaew,Piveteau,F,1989-08-24


<span style='font-family:Segoe UI'> **Cantidad de hombres y mujeres**

In [76]:
select if(gender like 'M', 'Hombres', 'Mujeres') as 'Empleados',
       count(*) as 'Número'
from employees
group by gender;

Empleados,Número
Hombres,179973
Mujeres,120051


<span style='font-family:Segoe UI'> **Cargos existentes en la empresa**

In [80]:
select distinct title as 'Cargo' from titles;

Cargo
Senior Engineer
Staff
Engineer
Senior Staff
Assistant Engineer
Technique Leader
Manager


<span style='font-family:Segoe UI'> **Salario promedio por cargo**

In [83]:
select Cargo, `Salario promedio`
from 
(  select title as 'Cargo',
          avg(salary) as 'Salario promedio'
   from employees
   join titles using(emp_no)
   join salaries using(emp_no)
   group by title
) as t1
order by `Salario promedio`;

Cargo,Salario promedio
Technique Leader,59294.3742
Assistant Engineer,59304.9863
Engineer,59508.0397
Senior Engineer,60543.2191
Manager,66924.2706
Staff,69309.1023
Senior Staff,70470.8353


<span style='font-family:Segoe UI'> **Salario promedio hombres y mujeres**

In [85]:
select if(gender like 'M', 'Masculino', 'Femenino') as 'Sexo',
       avg(salary) as 'Salario'
from employees
join salaries using(emp_no)
group by gender;

Sexo,Salario
Masculino,63838.1769
Femenino,63769.6032


## <span style='font-family:Segoe UI'> **2. Procedimientos, Triggers y Funciones**

<span style='font-family:Segoe UI'> Las bases de datos  que almacenan procedimientos y funciones son **employees** y **sakila**; sin embargo **sakila** contiene ejemplos de triggers.
    
## <span style='font-family:Segoe UI'> **2.1 Procedimientos**

In [ ]:
use employees;

In [4]:
delimiter //

create procedure show_departments()
modifies sql data
begin
    DROP TABLE IF EXISTS department_max_date;
    DROP TABLE IF EXISTS department_people;
    CREATE TEMPORARY TABLE department_max_date
    (
        emp_no int not null primary key,
        dept_from_date date not null,
        dept_to_date  date not null, # bug#320513
        KEY (dept_from_date, dept_to_date)
    );
    INSERT INTO department_max_date
    SELECT
        emp_no, max(from_date), max(to_date)
    FROM
        dept_emp
    GROUP BY
        emp_no;

    CREATE TEMPORARY TABLE department_people
    (
        emp_no int not null,
        dept_no char(4) not null,
        primary key (emp_no, dept_no)
    );

    insert into department_people
    select dmd.emp_no, dept_no
    from
        department_max_date dmd
        inner join dept_emp de
            on dmd.dept_from_date=de.from_date
            and dmd.dept_to_date=de.to_date
            and dmd.emp_no=de.emp_no;
    SELECT
        dept_no,dept_name,manager, count(*)
        from v_full_departments
            inner join department_people using (dept_no)
        group by dept_no;
        # with rollup;
    DROP TABLE department_max_date;
    DROP TABLE department_people;
end //

delimiter ;

Query OK

In [ ]:
use sakila;

In [17]:
DELIMITER $$

CREATE PROCEDURE film_in_stock(IN p_film_id INT, IN p_store_id INT, OUT p_film_count INT)
READS SQL DATA
BEGIN
     SELECT inventory_id
     FROM inventory
     WHERE film_id = p_film_id
     AND store_id = p_store_id
     AND inventory_in_stock(inventory_id);

     SELECT COUNT(*)
     FROM inventory
     WHERE film_id = p_film_id
     AND store_id = p_store_id
     AND inventory_in_stock(inventory_id)
     INTO p_film_count;
END $$

DELIMITER ;

Query OK

In [95]:
show procedure status where Db='sakila';

Db,Name,Type,Definer,Modified,Created,Security_type,Comment,character_set_client,collation_connection,Database Collation
sakila,film_in_stock,PROCEDURE,bibi@localhost,2021-04-15 23:05:34,2021-04-15 23:05:34,DEFINER,,utf8mb4,utf8mb4_general_ci,utf8mb4_general_ci
sakila,film_not_in_stock,PROCEDURE,bibi@localhost,2021-04-15 23:05:34,2021-04-15 23:05:34,DEFINER,,utf8mb4,utf8mb4_general_ci,utf8mb4_general_ci
sakila,rewards_report,PROCEDURE,bibi@localhost,2021-04-15 23:05:34,2021-04-15 23:05:34,DEFINER,Provides a customizable report on best customers,utf8mb4,utf8mb4_general_ci,utf8mb4_general_ci


## <span style='font-family:Segoe UI'> **2.2 Funciones**

In [23]:
show function status where Db='sakila';

Db,Name,Type,Definer,Modified,Created,Security_type,Comment,character_set_client,collation_connection,Database Collation
sakila,get_customer_balance,FUNCTION,bibi@localhost,2021-04-15 23:38:49,2021-04-15 23:38:49,DEFINER,,utf8mb4,utf8mb4_general_ci,utf8mb4_general_ci
sakila,inventory_held_by_customer,FUNCTION,bibi@localhost,2021-04-15 23:38:49,2021-04-15 23:38:49,DEFINER,,utf8mb4,utf8mb4_general_ci,utf8mb4_general_ci
sakila,inventory_in_stock,FUNCTION,bibi@localhost,2021-04-15 23:38:49,2021-04-15 23:38:49,DEFINER,,utf8mb4,utf8mb4_general_ci,utf8mb4_general_ci


In [26]:
DELIMITER $$

CREATE FUNCTION get_customer_balance(p_customer_id INT, p_effective_date DATETIME) RETURNS DECIMAL(5,2)
    DETERMINISTIC
    READS SQL DATA
BEGIN

       #OK, WE NEED TO CALCULATE THE CURRENT BALANCE GIVEN A CUSTOMER_ID AND A DATE
       #THAT WE WANT THE BALANCE TO BE EFFECTIVE FOR. THE BALANCE IS:
       #   1) RENTAL FEES FOR ALL PREVIOUS RENTALS
       #   2) ONE DOLLAR FOR EVERY DAY THE PREVIOUS RENTALS ARE OVERDUE
       #   3) IF A FILM IS MORE THAN RENTAL_DURATION * 2 OVERDUE, CHARGE THE REPLACEMENT_COST
       #   4) SUBTRACT ALL PAYMENTS MADE BEFORE THE DATE SPECIFIED

  DECLARE v_rentfees DECIMAL(5,2); #FEES PAID TO RENT THE VIDEOS INITIALLY
  DECLARE v_overfees INTEGER;      #LATE FEES FOR PRIOR RENTALS
  DECLARE v_payments DECIMAL(5,2); #SUM OF PAYMENTS MADE PREVIOUSLY

  SELECT IFNULL(SUM(film.rental_rate),0) INTO v_rentfees
    FROM film, inventory, rental
    WHERE film.film_id = inventory.film_id
      AND inventory.inventory_id = rental.inventory_id
      AND rental.rental_date <= p_effective_date
      AND rental.customer_id = p_customer_id;

  SELECT IFNULL(SUM(IF((TO_DAYS(rental.return_date) - TO_DAYS(rental.rental_date)) > film.rental_duration,
        ((TO_DAYS(rental.return_date) - TO_DAYS(rental.rental_date)) - film.rental_duration),0)),0) INTO v_overfees
    FROM rental, inventory, film
    WHERE film.film_id = inventory.film_id
      AND inventory.inventory_id = rental.inventory_id
      AND rental.rental_date <= p_effective_date
      AND rental.customer_id = p_customer_id;


  SELECT IFNULL(SUM(payment.amount),0) INTO v_payments
    FROM payment

    WHERE payment.payment_date <= p_effective_date
    AND payment.customer_id = p_customer_id;

  RETURN v_rentfees + v_overfees - v_payments;
END $$

DELIMITER ;

Db,Name,Type,Definer,Modified,Created,Security_type,Comment,character_set_client,collation_connection,Database Collation
sakila,get_customer_balance,FUNCTION,bibi@localhost,2021-04-15 23:38:49,2021-04-15 23:38:49,DEFINER,,utf8mb4,utf8mb4_general_ci,utf8mb4_general_ci
sakila,inventory_held_by_customer,FUNCTION,bibi@localhost,2021-04-15 23:38:49,2021-04-15 23:38:49,DEFINER,,utf8mb4,utf8mb4_general_ci,utf8mb4_general_ci
sakila,inventory_in_stock,FUNCTION,bibi@localhost,2021-04-15 23:38:49,2021-04-15 23:38:49,DEFINER,,utf8mb4,utf8mb4_general_ci,utf8mb4_general_ci


## <span style='font-family:Segoe UI'> **2.3 Triggers**

In [32]:
show triggers from sakila;

Trigger,Event,Table,Statement,Timing,Created,sql_mode,Definer,character_set_client,collation_connection,Database Collation
customer_create_date,INSERT,customer,SET NEW.create_date = NOW(),BEFORE,2021-04-15 23:38:55.59,"STRICT_TRANS_TABLES,STRICT_ALL_TABLES,NO_ZERO_IN_DATE,NO_ZERO_DATE,ERROR_FOR_DIVISION_BY_ZERO,TRADITIONAL,NO_AUTO_CREATE_USER,NO_ENGINE_SUBSTITUTION",bibi@localhost,utf8mb4,utf8mb4_general_ci,utf8mb4_general_ci
ins_film,INSERT,film,"BEGIN INSERT INTO film_text (film_id, title, description) VALUES (new.film_id, new.title, new.description); END",AFTER,2021-04-15 23:38:49.22,"STRICT_TRANS_TABLES,STRICT_ALL_TABLES,NO_ZERO_IN_DATE,NO_ZERO_DATE,ERROR_FOR_DIVISION_BY_ZERO,TRADITIONAL,NO_AUTO_CREATE_USER,NO_ENGINE_SUBSTITUTION",bibi@localhost,utf8mb4,utf8mb4_general_ci,utf8mb4_general_ci
upd_film,UPDATE,film,"BEGIN IF (old.title != new.title) OR (old.description != new.description) OR (old.film_id != new.film_id) THEN UPDATE film_text SET title=new.title, description=new.description, film_id=new.film_id WHERE film_id=old.film_id; END IF; END",AFTER,2021-04-15 23:38:49.23,"STRICT_TRANS_TABLES,STRICT_ALL_TABLES,NO_ZERO_IN_DATE,NO_ZERO_DATE,ERROR_FOR_DIVISION_BY_ZERO,TRADITIONAL,NO_AUTO_CREATE_USER,NO_ENGINE_SUBSTITUTION",bibi@localhost,utf8mb4,utf8mb4_general_ci,utf8mb4_general_ci
del_film,DELETE,film,BEGIN DELETE FROM film_text WHERE film_id = old.film_id; END,AFTER,2021-04-15 23:38:49.23,"STRICT_TRANS_TABLES,STRICT_ALL_TABLES,NO_ZERO_IN_DATE,NO_ZERO_DATE,ERROR_FOR_DIVISION_BY_ZERO,TRADITIONAL,NO_AUTO_CREATE_USER,NO_ENGINE_SUBSTITUTION",bibi@localhost,utf8mb4,utf8mb4_general_ci,utf8mb4_general_ci
payment_date,INSERT,payment,SET NEW.payment_date = NOW(),BEFORE,2021-04-15 23:38:56.61,"STRICT_TRANS_TABLES,STRICT_ALL_TABLES,NO_ZERO_IN_DATE,NO_ZERO_DATE,ERROR_FOR_DIVISION_BY_ZERO,TRADITIONAL,NO_AUTO_CREATE_USER,NO_ENGINE_SUBSTITUTION",bibi@localhost,utf8mb4,utf8mb4_general_ci,utf8mb4_general_ci
rental_date,INSERT,rental,SET NEW.rental_date = NOW(),BEFORE,2021-04-15 23:38:57.30,"STRICT_TRANS_TABLES,STRICT_ALL_TABLES,NO_ZERO_IN_DATE,NO_ZERO_DATE,ERROR_FOR_DIVISION_BY_ZERO,TRADITIONAL,NO_AUTO_CREATE_USER,NO_ENGINE_SUBSTITUTION",bibi@localhost,utf8mb4,utf8mb4_general_ci,utf8mb4_general_ci


## <span style='font-family:Segoe UI'> **3. Análisis estadístico**

In [ ]:
#!pip install pandas
#!pip install sqlalchemy # ORM for databases
#!pip install ipython-sql # SQL magic function
#!pip install pyhive[mariadb] # DB driver library
#!pip install seaborn

In [28]:
import sqlalchemy
from sqlalchemy import create_engine
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

In [32]:
import mysql.connector as mariadb

mariadb_conexion = mariadb.connect(host='localhost', port='3306',
                                   user='bibi', password='1971', database='world')
cursor = mariadb_conexion.cursor()

In [37]:
result = "SELECT * FROM city limit 1;"
cursor.execute(result)
filas = cursor.fetchall()
print("Resultado consulta:")
for fila in filas:
   print(fila)

Resultado consulta:
(1, 'Kabul', 'AFG', 'Kabol', 1780000)


In [39]:
paises_continente = "Select Continent,count(*) as Num_paises,(count(*)/239)*100 as Perc_paises,sum(SurfaceArea) as SurfaceArea,sum(Population) as Population from country group by Continent"
cursor.execute(paises_continente)
filas = cursor.fetchall()
print("Resultado consulta:")
for fila in filas:
   print(fila)

Resultado consulta:
('Asia', 51, Decimal('21.3400'), Decimal('31881005.00'), Decimal('3705025700'))
('Europe', 46, Decimal('19.2500'), Decimal('23049133.90'), Decimal('730074600'))
('North America', 37, Decimal('15.4800'), Decimal('24214470.00'), Decimal('482993000'))
('Africa', 58, Decimal('24.2700'), Decimal('30250377.00'), Decimal('784475000'))
('Oceania', 28, Decimal('11.7200'), Decimal('8564294.00'), Decimal('30401150'))
('Antarctica', 5, Decimal('2.0900'), Decimal('13132101.00'), Decimal('0'))
('South America', 14, Decimal('5.8600'), Decimal('17864926.00'), Decimal('345780000'))


In [44]:
Continent_language ="SELECT y.Continent,y.Name,x.*  FROM countrylanguage x inner join country y on x.countrycode=y.code;"
cursor.execute(Continent_language)
filas = cursor.fetchall()
print("Resultado consulta:")
for fila in filas:
   print(fila)

Resultado consulta:
('North America', 'Aruba', 'ABW', 'Dutch', 'T', Decimal('5.3'))
('North America', 'Aruba', 'ABW', 'English', 'F', Decimal('9.5'))
('North America', 'Aruba', 'ABW', 'Papiamento', 'F', Decimal('76.7'))
('North America', 'Aruba', 'ABW', 'Spanish', 'F', Decimal('7.4'))
('Asia', 'Afghanistan', 'AFG', 'Balochi', 'F', Decimal('0.9'))
('Asia', 'Afghanistan', 'AFG', 'Dari', 'T', Decimal('32.1'))
('Asia', 'Afghanistan', 'AFG', 'Pashto', 'T', Decimal('52.4'))
('Asia', 'Afghanistan', 'AFG', 'Turkmenian', 'F', Decimal('1.9'))
('Asia', 'Afghanistan', 'AFG', 'Uzbek', 'F', Decimal('8.8'))
('Africa', 'Angola', 'AGO', 'Ambo', 'F', Decimal('2.4'))
('Africa', 'Angola', 'AGO', 'Chokwe', 'F', Decimal('4.2'))
('Africa', 'Angola', 'AGO', 'Kongo', 'F', Decimal('13.2'))
('Africa', 'Angola', 'AGO', 'Luchazi', 'F', Decimal('2.4'))
('Africa', 'Angola', 'AGO', 'Luimbe-nganguela', 'F', Decimal('5.4'))
('Africa', 'Angola', 'AGO', 'Luvale', 'F', Decimal('3.6'))
('Africa', 'Angola', 'AGO', 'Mbundu',